<a href="https://colab.research.google.com/github/ayushbaghel01/InvestX_asgnmnt/blob/main/assignment_3__220261_Ayush_Baghel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
#from datetime import datetime

**NOTE : All the code is same as that is provided in the google meet, The code for finding efficiency of the below model is included in the last code blocks.**

In [ ]:
df = pd.read_csv('Train.csv')
print(df.head())
df=df.head(150)


# Clubbing MACD and RSI indicator


df['MACD_line']= (df.price.ewm(span=5).mean() - df.price.ewm(span=13).mean())

signal_line = df.MACD_line.ewm(span = 6).mean()
gain = np.where( df.price.shift(1) - df.price >0, df.price.shift(1) -df.price, 0)
loss = np.where(df.price.shift(1) - df.price<0, df.price.shift(1) - df.price, 0)
df['total_revenue']= 100 - (100/1+abs(gain/loss))
df['liabilities']=signal_line
df['equity']= df['MACD_line']
df['dividend'] = df.price.ewm(span=20).mean()
df['total_revenue']=(df.MACD_line + signal_line)*df['dividend']
# df['total_assets']= df.price.ewm(span =50).mean()
# df['share_outstanding']= signal_line - df.total_assets

         Date    Stock  liabilities      equity  total_assets  current_assets  \
0  2006-07-31  Stock 1   625.898686  925.086518   1550.985204      675.789905   
1  2006-08-31  Stock 1   407.387467  903.249093   1310.636561     1023.110983   
2  2006-09-29  Stock 1   464.069891  602.668832   1066.738722      709.711673   
3  2006-10-31  Stock 1   855.854937  391.375369   1247.230306      649.979564   
4  2006-11-30  Stock 1   453.954969  956.490566   1410.445534     1115.594551   

   current_liabilities  total_revenue  net_income   dividend  \
0           555.931329     548.488451  251.282553  68.184030   
1          1295.186240     513.446432  207.576940  42.232672   
2           598.719338     503.519871  133.719700  28.835722   
3           733.594339     596.006397  -76.881818   0.000000   
4          1152.274711     965.939247 -309.204301   0.000000   

   shares_outstanding       price  
0             1040831  100.000000  
1             1046649  101.924169  
2             109415

<ipython-input-631-49f1de2a111b>:14: RuntimeWarning: divide by zero encountered in true_divide
  df['total_revenue']= 100 - (100/1+abs(gain/loss))
<ipython-input-631-49f1de2a111b>:14: RuntimeWarning: invalid value encountered in true_divide
  df['total_revenue']= 100 - (100/1+abs(gain/loss))


In [ ]:
cols = list(df)[2:12]
print(cols)

['liabilities', 'equity', 'total_assets', 'current_assets', 'current_liabilities', 'total_revenue', 'net_income', 'dividend', 'shares_outstanding', 'price']


In [ ]:
df_for_training = df[cols].astype(float)

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [ ]:
trainX = []
trainY = []

In [ ]:
n_future = 1  # Number of days we want to look into the future based on the past days.
n_past = 14 # Number of past days we want to use to predict the future.

In [ ]:
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 9])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (136, 14, 10).
trainY shape == (136, 1).


In [ ]:
df_for_training_scaled;

In [ ]:
trainY;

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True)) 
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_58 (LSTM)              (None, 14, 64)            19200     
                                                                 
 lstm_59 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_29 (Dropout)        (None, 32)                0         
                                                                 
 dense_29 (Dense)            (None, 1)                 33        
                                                                 
Total params: 31,649
Trainable params: 31,649
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(trainX, trainY, epochs=5, batch_size=3, validation_split=0.1, verbose=1)

Epoch 1/5
41/41 [==============================] - 4s 27ms/step - loss: 0.2943 - val_loss: 0.0702
Epoch 2/5
41/41 [==============================] - 1s 19ms/step - loss: 0.1085 - val_loss: 0.0658
Epoch 3/5
41/41 [==============================] - 1s 19ms/step - loss: 0.0577 - val_loss: 0.0625
Epoch 4/5
41/41 [==============================] - 1s 19ms/step - loss: 0.0809 - val_loss: 0.0681
Epoch 5/5
41/41 [==============================] - 1s 20ms/step - loss: 0.0777 - val_loss: 0.0613


In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [ ]:
 df2= df.head(150)
train_dates = pd.to_datetime(df2['Date'])

In [ ]:
n_past = 1
n_days_for_prediction=50  #let us predict past 15 days

predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_days_for_prediction, freq='M').tolist()
print(predict_period_dates)

[Timestamp('2018-12-31 00:00:00', freq='M'), Timestamp('2019-01-31 00:00:00', freq='M'), Timestamp('2019-02-28 00:00:00', freq='M'), Timestamp('2019-03-31 00:00:00', freq='M'), Timestamp('2019-04-30 00:00:00', freq='M'), Timestamp('2019-05-31 00:00:00', freq='M'), Timestamp('2019-06-30 00:00:00', freq='M'), Timestamp('2019-07-31 00:00:00', freq='M'), Timestamp('2019-08-31 00:00:00', freq='M'), Timestamp('2019-09-30 00:00:00', freq='M'), Timestamp('2019-10-31 00:00:00', freq='M'), Timestamp('2019-11-30 00:00:00', freq='M'), Timestamp('2019-12-31 00:00:00', freq='M'), Timestamp('2020-01-31 00:00:00', freq='M'), Timestamp('2020-02-29 00:00:00', freq='M'), Timestamp('2020-03-31 00:00:00', freq='M'), Timestamp('2020-04-30 00:00:00', freq='M'), Timestamp('2020-05-31 00:00:00', freq='M'), Timestamp('2020-06-30 00:00:00', freq='M'), Timestamp('2020-07-31 00:00:00', freq='M'), Timestamp('2020-08-31 00:00:00', freq='M'), Timestamp('2020-09-30 00:00:00', freq='M'), Timestamp('2020-10-31 00:00:00'

In [ ]:
prediction = model.predict(trainX[-n_days_for_prediction:]) 

2/2 [==============================] - 0s 10ms/step


In [ ]:
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,9]

In [ ]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i.date())
    
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'price':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])

In [ ]:
original = df[['Date', 'price']]
original['Date']=pd.to_datetime(original['Date'])


<ipython-input-648-d6bbd2f4d398>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original['Date']=pd.to_datetime(original['Date'])


In [ ]:
df_forecast
df_ = pd.read_csv('Train.csv')
df_=df_.head(200)
predicted_price = df_forecast.price

In [ ]:
actual_price = df_.price[150:200]

In [ ]:
l=[]
for i in range(50):
  l.append(abs(predicted_price[i]-actual_price[150+i])/actual_price[150+i])
deviation_rd=np.array(l)
rms = deviation_rd.mean()
efficiency = (1-rms)*100
print(efficiency)
print(l)

84.51691416958795
[0.25087509155273435, 0.2198069575449498, 0.2028412629055392, 0.2110593636082106, 0.21501508464514288, 0.22762325971721206, 0.22392048020837188, 0.22881897614524488, 0.2178525155298794, 0.20804071317306363, 0.19635286224259912, 0.17672791056976964, 0.15633647284259336, 0.14271760166416592, 0.1400837586364302, 0.1403154269723265, 0.13605460616844703, 0.12331766745270496, 0.1294509445442731, 0.1398576762676146, 0.1356028936248688, 0.1390658423511951, 0.15149911913514172, 0.1484478172722429, 0.13434019366392866, 0.11569422588720216, 0.117287464782826, 0.1307930577446073, 0.12313706459243774, 0.11207233037086896, 0.11712723609695251, 0.108118015426089, 0.11616953568045035, 0.10881596755290175, 0.11870673658826073, 0.11467253943252143, 0.11073211152156534, 0.09943377439839315, 0.10855559412199976, 0.12454888290950689, 0.13482516394203342, 0.13086321475470003, 0.1285868835947129, 0.14023797867198687, 0.15160218075439216, 0.16042620191120702, 0.17831609810806473, 0.194423113

The model efficiency is increased from 82.2 to 84.5%